In [1]:
import sys
import os
import json
import re
import random

In [3]:
"""Azure Cosmos  API"""
from azure.cosmos import CosmosClient, exceptions, PartitionKey

"""Import azure command line interface wrappers for cosmos"""
from azure_cli import getcosmosaccountaccesskey, getcosmosaccounturl

In [4]:
""""Get the URL and key for the cosmos account"""

cosmosurl = getcosmosaccounturl(
    cosmosaccountname='raj-cosmos-2020',
    resourcegroup='azurelearning',
    subscription='1e6e9ea7-c0f7-4d7c-bae3-1199a177e943'
    ).strip('"')

In [5]:
accesskey = getcosmosaccountaccesskey(
    cosmosaccountname='raj-cosmos-2020',
    resourcegroup='azurelearning',
    subscription='1e6e9ea7-c0f7-4d7c-bae3-1199a177e943'
    )

In [6]:
"""Create a cosmos client"""
cosmos_client = CosmosClient(
    url=cosmosurl, 
    credential=accesskey)

In [7]:
#Get the list of databases in the cosmos account
databases = [db['id'] for db in cosmos_client.list_databases()]
print("Databases available in this cosmos account are {}".format(databases))

Databases available in this cosmos account are ['Products', 'demodb', 'cosmos-poc']


In [8]:
#Create database in cosmos account if it does not exists
cosmos_client.create_database_if_not_exists('cosmos-poc')

<DatabaseProxy [dbs/cosmos-poc]>

In [9]:
#get a database client to work with database
database_client = cosmos_client.get_database_client('cosmos-poc')

In [10]:
#create a collection container if it does not exist by passing the collection name and partition key 
database_client.create_container_if_not_exists(
    'demodata', 
    PartitionKey('/product')
    )

<ContainerProxy [dbs/cosmos-poc/colls/demodata]>

In [11]:
#create a container client to work with documents
container_client = database_client.get_container_client(container='demodata')

In [12]:
"""create a document in the container"""

for i in range(100):
    doc = {
        'id' : 'Itemnumber{}'.format(i),
        'organization' : 'ecommerce'
    }
    container_client.upsert_item(doc)
print("Data has been populated in container demodata")

Data has been populated in container demodata


In [13]:
"""Query the 3 documents from the container"""

iterations=0
for item in container_client.query_items(
    query='select * from demodata', 
    enable_cross_partition_query=True
    ):
    iterations+=1
    if iterations > 3:
        break
    else:
        print(json.dumps(item, indent=True))

{
 "product": "electronics",
 "id": "1",
 "_rid": "M2tmAIlnGSoBAAAAAAAAAA==",
 "_self": "dbs/M2tmAA==/colls/M2tmAIlnGSo=/docs/M2tmAIlnGSoBAAAAAAAAAA==/",
 "_etag": "\"1c01260b-0000-0100-0000-5fe170c10000\"",
 "_attachments": "attachments/",
 "_ts": 1608609985
}
{
 "id": "Itemnumber0",
 "organization": "ecommerce",
 "_rid": "M2tmAIlnGSoCAAAAAAAAAA==",
 "_self": "dbs/M2tmAA==/colls/M2tmAIlnGSo=/docs/M2tmAIlnGSoCAAAAAAAAAA==/",
 "_etag": "\"26014076-0000-0100-0000-5fe1929c0000\"",
 "_attachments": "attachments/",
 "_ts": 1608618652
}
{
 "id": "Itemnumber1",
 "organization": "ecommerce",
 "_rid": "M2tmAIlnGSoDAAAAAAAAAA==",
 "_self": "dbs/M2tmAA==/colls/M2tmAIlnGSo=/docs/M2tmAIlnGSoDAAAAAAAAAA==/",
 "_etag": "\"26014476-0000-0100-0000-5fe1929c0000\"",
 "_attachments": "attachments/",
 "_ts": 1608618652
}


In [14]:
#create another collection
database_client.create_container_if_not_exists(
    'demodata2', 
    PartitionKey('/product')
    )

<ContainerProxy [dbs/cosmos-poc/colls/demodata2]>

In [15]:
#create client for the second container to work with
container_client2 = database_client.get_container_client(container='demodata2')

In [16]:
"""create a document in the container"""

for i in range(50):
    doc = {
        'id' : 'Itemnumberinsecondcontainer{}'.format(i),
        'organization' : 'pharmaceutical',
        'product':'medicineno{}'.format(i)
    }
    container_client2.upsert_item(doc)
print("Data has been populated in container demodata2")

Data has been populated in container demodata2


In [17]:
"""Delete data from a container and repopulate it
     While deleting data, the partition key needs to be provided for that partitcular item
"""

for item in container_client2.query_items(
    query='select * from demodata2', 
    enable_cross_partition_query=True
    ):
    container_client2.delete_item(item, partition_key='{}'.format(item['product']))
print("Items have been deleted from container demodata2")

Items have been deleted from container demodata2


In [18]:
"""Self join document use case"""

'Self join document use case'

In [19]:
#create another collection
database_client.create_container_if_not_exists(
    'demodata3', 
    PartitionKey('/electronics')
    )

<ContainerProxy [dbs/cosmos-poc/colls/demodata3]>

In [20]:
#create client for the second container to work with
container_client3 = database_client.get_container_client(container='demodata3')

In [21]:
# create a document in the container

for i in range(10):
    doc = {
        'id' : str(random.randint(1,1000)),
        'electronics' : 'TV-Model-{}'.format(i),
        'cost' : 50+(99*i),
        'customers' : {
            'electronics' : 'TV-Model-{}'.format(i),
            'Address' : 'Random address at Zip {}'.format(i),
            'Name' : 'Customer-{}'.format(i)
        }
    }
    container_client3.upsert_item(doc)
print("Data has been populated in container demodata3")

Data has been populated in container demodata3


In [22]:
# query data by joining subset of the document with self

query = """
select a.id, a.electronics, a.cost, b.Address, b.Name
from demodata3 a
join
a.customers b
where a.electronics = b.electronics
"""

In [23]:
for item  in container_client3.query_items(query, enable_cross_partition_query=True):
    print(json.dumps(item, indent=True))

{
 "id": "23",
 "electronics": "TV-Model-0",
 "cost": 50,
 "Address": "Random address at Zip 0",
 "Name": "Customer-0"
}
{
 "id": "44",
 "electronics": "TV-Model-1",
 "cost": 149,
 "Address": "Random address at Zip 1",
 "Name": "Customer-1"
}
{
 "id": "88",
 "electronics": "TV-Model-2",
 "cost": 248,
 "Address": "Random address at Zip 2",
 "Name": "Customer-2"
}
{
 "id": "78",
 "electronics": "TV-Model-3",
 "cost": 347,
 "Address": "Random address at Zip 3",
 "Name": "Customer-3"
}
{
 "id": "79",
 "electronics": "TV-Model-4",
 "cost": 446,
 "Address": "Random address at Zip 4",
 "Name": "Customer-4"
}
{
 "id": "39",
 "electronics": "TV-Model-5",
 "cost": 545,
 "Address": "Random address at Zip 5",
 "Name": "Customer-5"
}
{
 "id": "45",
 "electronics": "TV-Model-6",
 "cost": 644,
 "Address": "Random address at Zip 6",
 "Name": "Customer-6"
}
{
 "id": "30",
 "electronics": "TV-Model-7",
 "cost": 743,
 "Address": "Random address at Zip 7",
 "Name": "Customer-7"
}
{
 "id": "38",
 "electron